# Ctrl-FreeQ API Demo

This notebook demonstrates how to use the CtrlFreeQ API for quantum control optimization.
The API provides a simple interface to load configuration files and run CtrlFreeQ optimizations.


In [ ]:
# Import the CtrlFreeQ API
from ctrl_freeq.api import (
    load_config,
    run_from_config,
    load_single_qubit_config,
    load_single_qubit_multiple_config,
    load_two_qubit_config,
    load_two_qubit_multiple_config,
)

from ctrl_freeq.visualisation.plotter import process_and_plot

## 1. Single Qubit Optimization

Let's start with a simple single qubit optimization example.


In [ ]:
# Load single qubit configuration
single_qubit_api = load_single_qubit_config()

# Display configuration summary
print("Single Qubit Configuration:")
print("=" * 40)
print(single_qubit_api.get_config_summary())

In [ ]:
# Run the optimization
print("Running single qubit optimization...")
single_qubit_solution = single_qubit_api.run_optimization()
print(f"Optimization completed. Solution shape: {single_qubit_solution.shape}")
print(f"Solution: {single_qubit_solution}")
waveforms = process_and_plot(
    single_qubit_solution, single_qubit_api.parameters, show_plots=True
)

## 2. Single Qubit with Multiple Initial/Target States

Now let's try a more complex single qubit example with multiple initial and target states.


In [ ]:
# Load single qubit multiple states configuration
single_multiple_api = load_single_qubit_multiple_config()

# Display configuration summary
print("Single Qubit Multiple States Configuration:")
print("=" * 50)
print(single_multiple_api.get_config_summary())

In [ ]:
# Run the optimization
print("Running single qubit multiple states optimization...")
single_multiple_solution = single_multiple_api.run_optimization()
print(f"Optimization completed. Solution shape: {single_multiple_solution.shape}")
print(f"Solution: {single_multiple_solution}")
waveforms = process_and_plot(
    single_multiple_solution, single_multiple_api.parameters, show_plots=True
)

## 3. Two Qubit Optimization

Let's move to a two qubit system optimization.


In [ ]:
# Load two qubit configuration
two_qubit_api = load_two_qubit_config()

# Display configuration summary
print("Two Qubit Configuration:")
print("=" * 40)
print(two_qubit_api.get_config_summary())

In [ ]:
# Run the optimization
print("Running two qubit optimization...")
two_qubit_solution = two_qubit_api.run_optimization()
print(f"Optimization completed. Solution shape: {two_qubit_solution.shape}")
print(f"Solution: {two_qubit_solution}")
waveforms = process_and_plot(
    two_qubit_solution, two_qubit_api.parameters, show_plots=True
)

## 4. Two Qubit with Multiple Initial/Target States

Finally, let's try a more complex example with two qubits and multiple states.


In [ ]:
# Load two qubit multiple states configuration
two_multiple_api = load_two_qubit_multiple_config()

# Display configuration summary
print("Two Qubit Multiple States Configuration:")
print("=" * 50)
print(two_multiple_api.get_config_summary())

In [ ]:
# Run the optimization
print("Running two qubit multiple states optimization...")
two_multiple_solution = two_multiple_api.run_optimization()
print(f"Optimization completed. Solution shape: {two_multiple_solution.shape}")
print(f"Solution: {two_multiple_solution}")
waveforms = process_and_plot(
    two_multiple_solution, two_multiple_api.parameters, show_plots=True
)

## 5. Loading Custom Configurations

You can also load custom configuration files or create configurations programmatically.


In [ ]:
# Example: Load a configuration from a file path
custom_api = load_config(
    "../src/ctrl_freeq/data/json_input/single_qubit_parameters.json"
)
print("Custom loaded configuration:")
print(custom_api.get_config_summary())

In [ ]:
# Example: Modify parameters before running
custom_api.update_parameter("optimization.max_iter", 500)
custom_api.update_parameter("optimization.targ_fid", 0.995)

print("Modified configuration:")
print(custom_api.get_config_summary())

## 6. Direct Configuration Usage

For quick runs, you can use the `run_from_config` function directly.


In [ ]:
# Run directly from a configuration file
print("Running optimization directly from config file...")
direct_solution = run_from_config(
    "../src/ctrl_freeq/data/json_input/single_qubit_parameters.json"
)
print(f"Direct optimization completed. Solution: {direct_solution}")

## 7. Dissipative (Lindblad) Optimization

Ctrl-FreeQ supports dissipative quantum systems via the Lindblad master equation.
When `dissipation_mode` is set to `"dissipative"`, the evolution includes amplitude
damping (T1) and pure dephasing (T2) for each qubit. The space is automatically
set to Liouville (density matrix) mode.

The Lindblad collapse operators are:
- **Amplitude damping**: $L_1 = \sqrt{1/T_1}\, \sigma^-$ (energy relaxation)
- **Pure dephasing**: $L_2 = \sqrt{1/T_2 - 1/(2T_1)}\, \sigma_z/2$ (phase decoherence)

The physical constraint $T_2 \leq 2 T_1$ is enforced automatically.

In [ ]:
# Load the dissipative single qubit configuration
from ctrl_freeq.api import load_single_qubit_dissipative_config

dissipative_api = load_single_qubit_dissipative_config()

# Display configuration summary (includes T1/T2 info)
print("Dissipative Single Qubit Configuration:")
print("=" * 50)
print(dissipative_api.get_config_summary())

In [ ]:
# Run the dissipative optimization
print("Running dissipative single qubit optimization...")
dissipative_solution = dissipative_api.run_optimization()
print(f"Optimization completed. Solution shape: {dissipative_solution.shape}")
print(f"Final fidelity: {dissipative_api.parameters.final_fidelity:.4f}")
waveforms = process_and_plot(
    dissipative_solution, dissipative_api.parameters, show_plots=True
)

### 7b. Programmatic Dissipative Configuration

You can also enable dissipation programmatically by providing T1/T2 values
and setting `dissipation_mode` in a configuration dictionary.

In [ ]:
# You can also take any existing config and make it dissipative programmatically
from ctrl_freeq.api import CtrlFreeQAPI

dissipative_config = {
    "qubits": ["q1"],
    "parameters": {
        "Delta": [10e6],
        "sigma_Delta": [0.0],
        "Omega_R_max": [40e6],
        "pulse_duration": [200e-9],
        "point_in_pulse": [100],
        "wf_type": ["cheb"],
        "wf_mode": ["cart"],
        "amplitude_envelope": ["gn"],
        "amplitude_order": [1],
        "coverage": ["single"],
        "sw": [5e6],
        "pulse_offset": [0.0],
        "pulse_bandwidth": [5e5],
        "ratio_factor": [0.5],
        "sigma_Omega_R_max": [0.0],
        "profile_order": [2],
        "n_para": [16],
        "J": [[0.0]],
        # Dissipation parameters
        "T1": [1e-3],  # 1 ms amplitude damping
        "T2": [500e-6],  # 500 us dephasing
    },
    "initial_states": [["Z"]],
    "target_states": {"Axis": [["-Z"]]},
    "optimization": {
        "space": "liouville",
        "dissipation_mode": "dissipative",
        "H0_snapshots": 1,
        "Omega_R_snapshots": 1,
        "algorithm": "l-bfgs",
        "max_iter": 200,
        "targ_fid": 0.99,
    },
}

api = CtrlFreeQAPI(dissipative_config)
print(api.get_config_summary())

In [ ]:
# Run the dissipative optimization
print("Running dissipative single qubit optimization...")
dissipative_solution = api.run_optimization()
print(f"Optimization completed. Solution shape: {dissipative_solution.shape}")
print(f"Final fidelity: {api.parameters.final_fidelity:.4f}")
waveforms = process_and_plot(dissipative_solution, api.parameters, show_plots=True)